<a href="https://colab.research.google.com/github/Chadliu123/TibameAI/blob/main/News.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 查看my drive
import glob
glob.glob("drive/My Drive/*")

In [ ]:
# 程式碼解壓縮
import zipfile
path = "drive/My Drive/chinese_news_test.zip"
f = zipfile.ZipFile(path)
f.extractall()
path = "drive/My Drive/chinese_news_trans.zip"
f = zipfile.ZipFile(path)
f.extractall()

In [ ]:
import os
import pandas as pd

def read_data(base):
  contents,targets = [],[]
  basedn = os.path.join(base,"*")
  for dn in glob.glob(basedn):
    upper = os.path.join(dn,"*.TXT")
    lower = os.path.join(dn,"*.txt")
    for fn in glob.glob(upper) + glob.glob(lower):
      with open(fn,"r",encoding="utf-8") as f:
        contents.append(f.read())
      target = os.path.split(dn)[-1]
      targets.append(target)
  df = pd.DataFrame({
      "content":contents,
      "target":targets
  })
  return df
traindf = read_data("chinese_news_trans")
testdf = read_data("chinese_news_test")
testdf

,content,target
0,日月光華 -- FDU_E.S.E.精華區文章閱讀\n-------------------...,環境
1,日月光華 -- FDU_E.S.E.精華區文章閱讀\n-------------------...,環境
2,日月光華 -- FDU_E.S.E.精華區文章閱讀\n-------------------...,環境
3,日月光華 -- FDU_E.S.E.精華區文章閱讀\n-------------------...,環境
4,日月光華 -- FDU_E.S.E.精華區文章閱讀\n-------------------...,環境
...,...,...
96,日月光華 -- Economics精華區文章閱讀\n--------------------...,經濟
97,日月光華 -- Economics精華區文章閱讀\n--------------------...,經濟
98,日月光華 -- Economics精華區文章閱讀\n--------------------...,經濟
99,日月光華 -- Economics精華區文章閱讀\n--------------------...,經濟


In [ ]:
# Series.replace({"李白":0})
# unique/value_counts
news = traindf["target"].unique()
trans = {w:i for i, w in enumerate(news)}
trans_r = {i:w for i, w in enumerate(news)}
y_train = traindf["target"].replace(trans)
y_test = testdf["target"].replace(trans)
y_test

0      0
1      0
2      0
3      0
4      0
      ..
96     9
97     9
98     9
99     9
100    9
Name: target, Length: 101, dtype: int64

In [ ]:
import jieba
from urllib.request import urlretrieve
if not os.path.exists("dict.txt.big"):
  # 載入jieba定義的字典
  url = "https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big"
  urlretrieve(url, "dict.txt.big")
jieba.set_dictionary("dict.txt.big")
# apply(函式名字)
def newscut(p):
    return " ".join(jieba.cut(p))
x_train = traindf["content"].apply(newscut)
x_test = testdf["content"].apply(newscut)
x_test

Building prefix dict from /content/dict.txt.big ...
Dumping model to file cache /tmp/jieba.u501edca284da514cb68b53a20324f4e3.cache
Loading model cost 1.559 seconds.
Prefix dict has been built successfully.


0      日月 光華   --   FDU _ E . S . E . 精華區 文章 閱讀 \n --...
1      日月 光華   --   FDU _ E . S . E . 精華區 文章 閱讀 \n --...
2      日月 光華   --   FDU _ E . S . E . 精華區 文章 閱讀 \n --...
3      日月 光華   --   FDU _ E . S . E . 精華區 文章 閱讀 \n --...
4      日月 光華   --   FDU _ E . S . E . 精華區 文章 閱讀 \n --...
                             ...                        
96     日月 光華   --   Economics 精華區 文章 閱讀 \n ----------...
97     日月 光華   --   Economics 精華區 文章 閱讀 \n ----------...
98     日月 光華   --   Economics 精華區 文章 閱讀 \n ----------...
99     日月 光華   --   Economics 精華區 文章 閱讀 \n ----------...
100    日月 光華   --   Economics 精華區 文章 閱讀 \n ----------...
Name: content, Length: 101, dtype: object

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
x_train_count = vec.fit_transform(x_train)
x_test_count = vec.transform(x_test)
print(x_train_count)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha=0.1)
clf.fit(x_train_count,y_train)

MultinomialNB(alpha=0.1, class_prior=None, fit_prior=True)

In [ ]:
from sklearn.metrics import accuracy_score
pre = clf.predict(x_test_count)
accuracy_score(pre,y_test)

1.0

In [ ]:
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(y_test,pre)
pd.DataFrame(mat,
      index=[name+"(原本)" for name in news],
      columns=[name+"(預測)" for name in news])

,環境(預測),體育(預測),交通(預測),醫藥(預測),軍事(預測),藝術(預測),計算機(預測),教育(預測),政治(預測),經濟(預測)
環境(原本),10,0,0,0,0,0,0,0,0,0
體育(原本),0,10,0,0,0,0,0,0,0,0
交通(原本),0,0,10,0,0,0,0,0,0,0
醫藥(原本),0,0,0,10,0,0,0,0,0,0
軍事(原本),0,0,0,0,10,0,0,0,0,0
藝術(原本),0,0,0,0,0,10,0,0,0,0
計算機(原本),0,0,0,0,0,0,10,0,0,0
教育(原本),0,0,0,0,0,0,0,10,0,0
政治(原本),0,0,0,0,0,0,0,0,11,0
經濟(原本),0,0,0,0,0,0,0,0,0,10


In [ ]:
p = input("輸入一首新聞:")
x_predict = vec.transform([newscut(p)])
proba = clf.predict_proba(x_predict)[0]
for w, p in zip(news, proba):
    print(w, ":", round(p, 4))

輸入一首新聞:川普確診
環境 : 0.0193
體育 : 0.0357
交通 : 0.0239
醫藥 : 0.812
軍事 : 0.0213
藝術 : 0.0125
計算機 : 0.0072
教育 : 0.0119
政治 : 0.0426
經濟 : 0.0137
